In [13]:
import os
import logging
from oauth2client.client import _raise_exception_for_reading_json

from google.oauth2 import service_account
import googleapiclient.discovery

from httplib2 import Http

from GoogleApiSupport import apis
import GoogleApiSupport.sheets as sheets

In [10]:
def get_service(api_name, service_credentials_path=None):
    service_credentials_path = get_service_credentials_path(service_credentials_path)

    service = None

    credentials = service_account.Credentials.from_service_account_file(
        service_credentials_path,
        scopes=apis.get_api_config(api_name)['scope']
    )

    service = googleapiclient.discovery.build(apis.get_api_config(api_name)['build'],
        apis.get_api_config(api_name)['version'],
        credentials=credentials
    )

    if not service:
        logging.error(' UNABLE TO RETRIEVE CREDENTIALS | Expected credential paths: ' + ', '.join(
            service_credentials_path) + ' | More info in project Documentation folder setup_credentials.md file')
    return service


def get_service_credentials_path(service_credentials_path=None):
    if service_credentials_path:
        service_credentials_path = service_credentials_path
        logging.info('Trying to use credentials from ' + 'Method 0: Path from function argument | ' + service_credentials_path)
    elif os.environ.get('GOOGLE_APPLICATION_CREDENTIALS'):
        service_credentials_path = os.environ['GOOGLE_APPLICATION_CREDENTIALS']
        logging.info('Trying to use credentials from ' + 'Method 1: Environment variable GOOGLE_APPLICATION_CREDENTIALS | ' + service_credentials_path)
    elif os.environ.get('SERVICE_CREDENTIALS_PATH'): ## TO DO: DELETE - DEPRECATED
        service_credentials_path = os.environ['SERVICE_CREDENTIALS_PATH']
        logging.warning('Trying to use credentials from ' +  'Method 2 (deprecated): Environment variable SERVICE_CREDENTIALS_PATH | ' + service_credentials_path)
    else:
        service_credentials_path = os.path.join(os.path.expanduser('~'), '.credentials', 'service_credentials.json')
        logging.info('Tying to use credentials from ' + 'Method 3: Default path for credentials ~/.credentials/service_credentials.json | ' + service_credentials_path + ' | Nor path passed neither environment variables, take a look into `docs/setup_credentials.md` file')      
      
    if (os.path.isfile(service_credentials_path)):
        logging.info('Found file credentials in' + service_credentials_path)
        return service_credentials_path
    else:
        raise Exception('UNABLE TO FIND CREDENTIALS FILE | Environment variable not defined or file from provided path does not exist | More info in project docs folder setup_credentials.md file')

def sheet_to_pandas(spreadsheetId, sheetName='', sheetRange='', index=''):
    '''
    PARAMETERS:
        service - Api service
        spreadsheetId - Id of the desired document
        sheetName - Name of the desired page 'Hoja1' (optional) (by default: first page)
        sheetRange - Range of the desired info 'A1:C6' (optional) (by default: WHOLE PAGE)
        index - column you want to be the index of the resulting dataframe (optional) (by default: none of the columns is set as index)
    '''
    service = get_service("sheets")
    if (sheetRange != ''):
        sheetRange = '!'+sheetRange

    newresult = service.spreadsheets().values().get(
        spreadsheetId=spreadsheetId,
        valueRenderOption='FORMATTED_VALUE',
        range=sheetName+sheetRange
    ).execute()

    headers = newresult['values'].pop(0)

    if (index == ''):
        return pd.DataFrame(newresult['values'], columns=headers)
    else:
        return pd.DataFrame(newresult['values'], columns=headers).set_index(index, drop=False)

ModuleNotFoundError: No module named 'GoogleApiSupport.sheeets'

In [15]:
a = sheets.sheet_to_pandas('1iViBL9BDdd6nr0pJSs', 'hoja1')

In [16]:
a

,url,hint
0,https://topodesigns.com/,None
1,https://www.peakdesign.com/products/packing-cube,packingcube
2,https://www.patagonia.com/product/lightweight-...,"socks, merino"
3,https://www.patagonia.com/product/stand-up-bel...,sling
4,https://packhacker.com/packing-list/,packs
5,https://matadorup.com/,None
6,https://huckberry.com/store/the-james-brand/ca...,pen
7,https://www.aersf.com/travel-kit-black/,travelkit
8,https://shop.outlier.nyc/shop/retail/,clothes
9,https://www.heimplanet.com/shop/Transit-Line-S...,sling


In [4]:
service = get_service("sheets")
request = service.spreadsheets().values().get(
        spreadsheetId='1iViBL9BD6xEtzudhzh9x-oGEQN1UfI5e1dd6nr0pJSs',
        valueRenderOption='FORMATTED_VALUE',
        range='hoja1'
    )

In [8]:
apis.get_api_config('sheets')['scope']

'https://www.googleapis.com/auth/spreadsheets'

In [ ]:
dir(service.sp)


In [ ]:
from oauth2client.service_account import ServiceAccountCredentials
from GoogleApiSupport import apis


service_credentials_path = auth.get_service_credentials_path(None)

credentials = ServiceAccountCredentials.from_json_keyfile_name(
        service_credentials_path,
        scopes=apis.get_api_config('sheets')['scope']
    )

In [ ]:
apis.api_configs['sheets']

In [ ]:
!pip install --upgrade oauth2client